# IMPORTACIONES Y DEFINICIONES

Importar librerías y definir funciones:
(En as_gray la conversión de RGB a 8bit tiene esa partición para intentar tomar los aguamarina que aparecen como blancos en lo posible)

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
@author: raulrica
"""

from scipy.optimize import curve_fit
from scipy import constants as cons

import numpy as np
import pandas as pd
from pandas import DataFrame, Series  # for convenience

import matplotlib as mpl
import matplotlib.pyplot as plt

import pims
import trackpy as tp


def fitf(x, a, b):
    return a*x**b

@pims.pipeline
def as_gray(frame):
    red = frame[:, :, 0]
    green = frame[:, :, 1]
    blue = frame[:, :, 2]
    return 0.2*red + 0.4*green + 0.4*blue 

# PARTE DE TUNEAR PARAMETROS

Leemos el video y mostramos el primer fotograma de forma interactiva para tener una estimación del tamaño (en pixeles) de las particulas

In [ ]:
vsrc = '/home/raulrica/Documentos/TESTS/Janus/agua 1.avi'

# Si la fuente es una serie de imagenes (.tif por ejemplo):

#vsrc = '/home/raulrica/Documentos/TESTS/Janus/*.tif'

frames = as_gray(pims.open(vsrc))  #Si la imagen es en RGB
#frames = as_gray(pims.open(vsrc)) #Si la imagen es en blanco y negro
%matplotlib notebook
plt.imshow(frames[0], cmap='gray')

Introducimos tamaño de particula (impar y mejor sobreestimado que subestimado)

Detectamos objetos en el primer frame y vemos distribucion de masa y tamaño para poder filtrar luego

In [ ]:
ps = 17

f = tp.locate(frames[0], ps, invert=True)

# Descomentar para enseñar foto con la deteccion preliminar sin filtrar

%matplotlib inline
#plt.figure()
#plt.title('Unfiltered')
#tp.annotate(f, frames[0])

plt.figure()
ax = plt.axes()
ax.hist(f['mass'], bins=50)
ax.set(title='mass distribution')

plt.figure()
ax = plt.axes()
ax.hist(f['size'], bins=50)
ax.set(title='size distribution')

#plt.figure()
#ax = plt.axes()
#ax.hist(f['ecc'], bins=50)
#ax.set(title='Eccentricity distribution')

Filtramos partículas por

- Brillo (masa)
- Tamaño
- Excentricidad
- ...

In [ ]:
minmass = 250

minsize = 3.5
maxsize = 5.0

#minecc  = 0.2
#maxecc  = 1e6

f = tp.locate(frames[0], ps, invert=True, minmass=minmass)
f = f.loc[((f['size'] > minsize) & (f['size'] < maxsize))]
#f = f.loc[((f['ecc'] > minecc) & (f['ecc'] < maxecc))]

%matplotlib notebook
plt.figure()
plt.title('Filtered')
tp.annotate(f, frames[0])

# PARTE BULKY Y LENTA

Con todos los parametros en cuenta analizamos la parte del video que nos interese (sf = 0 ; ef = -1 para video completo)

El parametro mem será el que determine cuantos fotogramas puede estar perdida una partícula antes de reaparecer para ser considerada como otra distinta.

También lo usaremos para filtrar trayectorias de partículas que hayan permanecido en el video durante menos de mem fotogramas

In [ ]:
sf  = 0
ef  = 50
mem = 30 

f = tp.batch(frames[sf:ef], ps, minmass=minmass, invert=True)
f = f.loc[((f['size'] > minsize) & (f['size'] < maxsize))]
#f = f.loc[((f['ecc'] > minecc) & (f['ecc'] < maxecc))]

t = tp.link(f, int(ps/2), memory=mem)

plt.figure()
plt.title('All trajectories')
tp.plot_traj(t)

t = tp.filter_stubs(t, mem)

plt.figure()
plt.title('Persistent Trajectories')
tp.plot_traj(t)

# POSTPROCESADO DE TRAYECTORIAS

Introducimos los datos para calcular MSDs de las partículas

In [ ]:
mpp = 1/15  #microns per pixel
fps = 14    #frames per second

In [ ]:
im = tp.imsd(t, mpp, fps)

%matplotlib inline
fig, ax = plt.subplots()
for col in im.columns:
    ax.plot(im[col], alpha=0.1)
ax.set(ylabel=r'$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]',
       xlabel='lag time $t$')
ax.set_xscale('log')
ax.set_yscale('log')

In [ ]:
em  = tp.emsd(t, mpp, fps)
dt  = em.index.values
msd = em[:]

params, covar = curve_fit(fitf, dt, msd, p0 = (1, 1))
dtext = np.arange(np.min(dt),np.max(dt),0.001)
perr = np.sum(np.trace(covar))

fig, ax = plt.subplots()
ax.plot(em.index, em, 'bo', alpha = 0.1)
ax.plot(dtext,fitf(dtext, *params),'k-')  
ax.set(ylabel = r'$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]',
       xlabel = 'lag time $t$',
       title  = 'Ensemble MSD')
ax.set_xscale('log')
ax.set_yscale('log')

Calculos extra

In [ ]:
A    = params[0]
D    = 1e-12*A/4
dia  = 1e-6      #m
r    = dia/2     #m
T    = 288       #K 

visc = cons.k*T/(6*cons.pi*D*r)

print('Viscosity:',visc*1000,'cps')